In [11]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import json

In [3]:
session = HTMLSession()

In [5]:
artLinks = json.loads(open('artworks.json', 'r').read())

In [25]:
links = artLinks["SAAM"]

In [28]:
artworkResp = [session.get(link) for link in links]

In [29]:
def extractFromSmithsonian(res):
    if not res.ok: return
    soup = BeautifulSoup(res.content)
    try:
        table = soup.select("#page > div > div.row > section > div.region.region-content > article > div > div.about-artwork.row > div.tombstone.col-sm-12.col-md-6")[0]
        heading = table.select("dt")
        content = table.select("dd")
        info = {}
        for i in range(len(heading)):
            info[heading[i].get_text()] = content[i].get_text().strip()
        imgAddress = soup.select_one("div.active img, #primary-artwork-image > img").get("src")
        pieceMapped = {"Title": info["Title"], "Artist": info["Artist"], 
                       "Year":info["Date"], "Gallery": "Smithsonian American Art Museum", 
                       "Collection": info["Location"].split("\n ")[-3], "Image_Address": imgAddress}
        return pieceMapped
    except:
        print(res, res.url)
    

In [35]:
smithsonianExtracts = [extractFromSmithsonian(resp) for resp in artworkResp]